In [8]:
from infrastructure.database.mongo_client import MongoDBClient
from infrastructure.repository import YouTubeContentRepository
from application.use_case import YouTubeContentDownloadAndStore

import os
from dotenv import load_dotenv

load_dotenv()

# MongoDB 클라이언트 초기화
client = MongoDBClient(uri=os.environ['MONGO_CONNECTION_STRING'])
client.connect()

# 저장소 초기화
repository = YouTubeContentRepository(client)

# 유즈케이스 초기화
yds = YouTubeContentDownloadAndStore(repository)

Connected to MongoDB


In [9]:
test_url = 'https://www.youtube.com/watch?v=MmW166cHj54&list=PLO-mt5Iu5TeZF8xMHqtT_DhAPKmjF6i3x&index=1'
test_category = 'test'

from domain.value_object import YouTubeVideoLink
repository.delete_by_url(YouTubeVideoLink(test_url))

custom_download_folder = '../downloads'
yds.execute(test_url, test_category, custom_download_folder)

데이터 저장 완료
../downloads/test
파일이 이미 존재합니다: ../downloads/test/유니티 뱀서라이크 개발강좌가 온다?! 언데드 서바이버.mp3


True

In [10]:
import pandas as pd

youtube_parse_list = pd.read_csv('../data/youtube_parse_list.csv')
youtube_parse_list

,category,url,noplaylist
0,뱀서라이크,https://www.youtube.com/playlist?list=PLO-mt5I...,False
1,비행기슈팅,https://www.youtube.com/playlist?list=PLO-mt5I...,False
2,탑다운RPG,https://www.youtube.com/playlist?list=PLO-mt5I...,False
3,플랫포머,https://www.youtube.com/playlist?list=PLO-mt5I...,False
4,유니티6,https://www.youtube.com/watch?v=350EWYC_1ZA&li...,True
5,유니티6,https://www.youtube.com/watch?v=Q7mHfbEzaJM&li...,True
6,유니티6,https://www.youtube.com/watch?v=r4aZypBrFHE,True
7,유니티6,https://www.youtube.com/watch?v=3mSO1sMdGxM,True


In [11]:
import yt_dlp

def get_playlist_info(url):
    ydl_opts = {
        'quiet': True,
        'extract_flat': True,  # 재생목록일 경우 정보만 가져옴
        'noplaylist': False,  # 사용자의 선택에 따라 재생목록 전체 또는 단일 동영상 처리
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=False)
        return info


In [14]:
custom_download_folder = '../downloads'

for row in youtube_parse_list.itertuples(index=False):
    category = row.category
    url = row.url
    is_playlist = not row.noplaylist

    if is_playlist:
        playlist_info = get_playlist_info(url)
        for entry in playlist_info['entries']:
            video_url = entry.get('url')
            yds.execute(video_url, category, custom_download_folder)
            print('----' * 7)

    else:
        yds.execute(url, category, custom_download_folder)
        print('----' * 7)

데이터가 이미 존재합니다.
../downloads/test
파일이 이미 존재합니다: ../downloads/test/유니티 뱀서라이크 개발강좌가 온다?! 언데드 서바이버.mp3
--------------------------------------------------------
데이터 저장 완료
../downloads/뱀서라이크
오디오 다운로드 중: https://www.youtube.com/watch?v=qOTbP9ciJ88
   오디오가 성공적으로 다운로드되어 저장되었습니다: ../downloads/뱀서라이크/2D 오브젝트🧍만들기 [유니티 뱀서라이크 01].mp3
--------------------------------------------------------
데이터 저장 완료
../downloads/뱀서라이크
오디오 다운로드 중: https://www.youtube.com/watch?v=YAu4yWU5D5U
   

KeyboardInterrupt: 